## Data Transactions Words Cleaning
### JJD-Predictors
Con el objetivo de terminar de limpiar el campo ref (el cual es una union de los campos ref1 y ref2) se eliminaran palabras que esta presentes menos de 10 veces dentro del campo ref. Esto se hace con el proposito de eliminar cadenas de texto sin sentido, nombres propios, direcciones e-mail, etc. Empezamos cargando los datos que ya se han limpiado en el Notebook "Data Transactions Cleaning"

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from unicodedata import normalize

%matplotlib inline

data_transactions = pd.read_pickle('./transactions_BC_preclean.pkl')

Ahora se crea un data frame las palabras presentes en el campo ref de todo el dataset. En total existen 384886 palabras distintas en el campo ref. Note que las palabras con pocas ocurrencias en el campo ref son palabras sin sentido.

In [2]:
a = np.array(data_transactions.ref.str.split())
a = a.flatten('F').ravel().tolist() 
flat_list = [item for sublist in a for item in sublist]
df = pd.DataFrame({"words":flat_list})
value_counts = df["words"].value_counts()
len(value_counts)

In [6]:
df["words"].value_counts()

pago                            3689975
factura                         1119288
pse                              607992
recarga                          593844
saldo                            528965
compra                           501709
medellin                         475635
express                          458625
esp                              433992
publicas                         416629
empresas                         416352
transaccion                      369826
certificados                     368950
pagos                            321327
postpago                         306224
credito                          305436
generacion                       299130
libertad                         283066
tradicion                        282866
contrato:                        277174
referencia:                      276886
valor:                           265602
traves                           245398
ref                              231394
cpv                              229042


Se escogen las palabras a eliminar, como las palabras que aparecen menos de 10 veces en todo el campo ref. Eliminando estas palabras quedaran 13432 palabras distintas en el campo ref, para todo el dataset. Ademas, palabras como pago y factura se eliminan porque tienen alta frecuencia y no seran utiles para la clasificacion. De hecho estas palabras pueden generar un sesgo en el modelo.

In [6]:
to_remove = value_counts[value_counts <= 10].index
to_remove= [str(x) for x in list(to_remove)]
len(value_counts) - len(to_remove)

13432

In [8]:
words = set(df["words"].unique()) - set(to_remove)
words = list(words - set(["pago" , "pagos", "factura", "facturas", "pago:" , "pagos:", "factura:", "facturas:"]))

In [10]:
data_transactions["ref"] = data_transactions["ref"].astype('str')
data_transactions["ref"] = data_transactions["ref"].apply(lambda x: ' '.join(str(e) for e in [word for word in str.split(x) if word in words]))
data_transactions["ref"] = data_transactions["ref"].apply(lambda x: x.strip()) 
data_transactions["ref"] = data_transactions["ref"].replace({"nan":np.nan, '':np.nan})

Una vez eliminadas las palabras deseadas se eliminan las transacciones que terminaron con el campo ref vacio al realizar este proceso. En este momento el dataset de transacciones queda con un total de 8'500 238, y se genera el dataset de transacciones que se usara para la clasificacion. 

In [11]:
data_transactions["ref"] = data_transactions["ref"].replace({"nan":np.nan, '':np.nan}) 
data_transactions = data_transactions[(~data_transactions['ref'].isnull())]

In [12]:
data_transactions = data_transactions.reset_index(drop=True)

85960570

In [13]:
data_transactions.to_pickle("./transactions_BC.pkl")
100*data_transactions.isnull().mean()

id_trn_ach       0.00000
id_cliente       0.00000
fecha            0.00000
hora             0.00000
valor_trx        0.00000
sector          66.10290
subsector       66.10290
descripcion     66.10290
descripcion2    99.81146
ref              0.00000
dtype: float64